#### Dependencies
*Install the following libraries before running the TDA pipeline.*

In [ ]:
install.packages(c("TDA", "Matrix"))

---------------------------

### Load libraries

In [ ]:
library("Matrix") #package for sparse matrices
library("TDA")

### Set working directory

In [ ]:
working_dir_path <- setwd("~/Documents/GitHub/TDA-Microscopy-Pipeline/TDA/general-pipeline")

### Load necessary functions

In [ ]:
source("utilities.R")

### Create a folder with a subfolder for each data group for storing saved tda computations
*You only need to run this block once*

In [ ]:
dir.create("saved_computations", showWarnings = FALSE)
saved_computations <- concat_path(working_dir_path, "saved_computations")

#choose names for two data groups
groups <- list("group_0", "group_25")

for (i in 1:length(groups)){
  dir.create(concat_path(saved_computations,groups[i]),  showWarnings = FALSE)
}

### Create a single csv file for storing necessary parameters (max birth and max death) obtained from all data groups
*You only need to run this block once; after running it once comment it out*

In [ ]:
max_values <- data.frame(matrix(ncol = 2, nrow = 0))
colnames(max_values) <- c("maxbirth", "maxdeath")  

write.csv(max_values, concat_path(saved_computations,"max-values.csv"), row.names=FALSE)

### Choose cell types

In [ ]:
#specify location path of the file with the cell type codes and their colors
cell_types_path <- "~/Documents/GitHub/TDA-Microscopy-Pipeline/Identification/cell_colors.csv"

#read all cell type codes from your designated file
cell_types_file <- read.csv(cell_types_path)
all_cell_types <- cell_types_file$Code

# indicate cell types for only including specific cells 
cell_types <- all_cell_types[2]   # this can either be a single value or a vector

index <- NaN    # default index for cell types is the last column

sprintf("Cell types used: %s", paste(cell_types, collapse=", "))

## I. Choose a data group and its location

In [ ]:
#choose a group folder where to save computations
group <- "group_0"

# specify location path of the csv files of that group
csv_dir_path <- "~/Documents/GitHub/TDA-Microscopy-Pipeline/Discretized-images/0"

## II. Compute persistence diagrams and save them
*We use Delaunay complex filtration to compute persistence diagrams*

In [ ]:
compute_PDs(csv_dir_path, group, saved_computations, cell_types, index)

## III. Determine the max birth and max death among all data files
*This is crucial for VI block and will ensure that all plots from both groups have the same scale*

In [ ]:
#Before running this block, compute persistence diagrams for all data groups (i.e choose another data group and specify its location in I-block, and compute its persistence diagrams in II-block) 

max_values <- read.csv(concat_path(saved_computations, "max-values.csv"))

max_birth <- max(max_values$maxbirth)
max_death <- max(max_values$maxdeath)

### Plot persistence diagrams

In [ ]:
save_plot <- TRUE #set FALSE if you don't want to save plots

plot_PDs(group, max_birth, max_death, saved_computations, save_plot)

### Plot representative cycles that persist (live) over a certain threshold

In [ ]:
#choose a persistence threshold
persist_threshold <- 20 

save_plot <- TRUE #set FALSE if you don't want to save plots

#plot representative cycles with persistence above the chosen threshold
plot_representative_cycles(csv_dir_path, cell_types, persist_threshold, group, saved_computations, save_plot)

## IV. Compute persistence landscapes and save them
*You need to have persistence diagrams computed first*

In [ ]:
#choose a discretization step (use the same for all groups)
discr_step <- 0.3

save_computations_csv <- FALSE #set TRUE if additionally to saving persistence landscapes in RData format you want to save each of them in a separate csv file. This option is provided if, for example, you plan to use persistence landscapes for further analysis in another programming language. Note that in a csv file row k corresponds to a persistence landscape function at depth k, where depth 1 corresponds to a outermost persistence landscape function, and column names correspond to x-values. 

compute_PLs(group, max_birth, max_death, discr_step, saved_computations, save_computations_csv)

### Plot persistence landscapes 

In [ ]:
save_plot <- TRUE #set FALSE if you don't want to save plots

plot_PLs(group, max_birth, max_death, discr_step, saved_computations, save_plot)

## V. Compute the average persistence landscape and save it
*You need to have persistence landscapes computed first*

In [ ]:
compute_avgPL(group, max_birth, max_death, discr_step, saved_computations)

### Plot the average persistence landscape

In [ ]:
save_plot <- TRUE #set FALSE if you don't want to save plots

plot_avgPL(group, max_birth, max_death, discr_step, saved_computations, save_plot)

## At this point, you need to have computed persistence diagrams, persistence landscapes, and average persistence landscapes for BOTH groups

-----------------

## VI. Plot the difference of average persistence landscapes of two data groups
*You need to have computed average persistence landscapes for both groups (make sure that they were plotted on the same scale)*

In [ ]:
save_plot <- TRUE #set FALSE if you don't want to save plots

#avgPL of groups[0] - avgPL of groups[1]
plot_avgPLs_difference(groups, max_birth, max_death, discr_step, saved_computations, save_plot) 

## VII. Perform a permutation test on persistence landscapes from two data groups
*You need to have computed persistence landscapes for both groups*

In [ ]:
#number of permutations
num_repeats <- 10000

permutation_test_for_PLs(groups, saved_computations, num_repeats)